# Helper functions

In [61]:
def readPixelConfig(ASIC, x, y):
    ASIC.CmdPrepForRead() #0000
    ASIC.PrepareMultiConfig() #8000
    bankToWrite = int(y/48);
    if (bankToWrite == 0):
     colToWrite = 0x700 + y%48;
    elif (bankToWrite == 1):
     colToWrite = 0x680 + y%48;
    elif (bankToWrite == 2):
     colToWrite = 0x580 + y%48;
    elif (bankToWrite == 3):
     colToWrite = 0x380 + y%48;
    else:
     print('unexpected bank number')
    ASIC.RowCounter.set(x) #6011
    ASIC.ColCounter.set(colToWrite) #6013
    readBack = ASIC.WritePixelData.get() #5000
    ASIC.CmdPrepForRead() #0000
    return readBack

def writedPixelConfig(ASIC, x, y, value):
    ASIC.CmdPrepForRead() #0000
    ASIC.PrepareMultiConfig() #8000
    bankToWrite = int(y/48);
    if (bankToWrite == 0):
     colToWrite = 0x700 + y%48;
    elif (bankToWrite == 1):
     colToWrite = 0x680 + y%48;
    elif (bankToWrite == 2):
     colToWrite = 0x580 + y%48;
    elif (bankToWrite == 3):
     colToWrite = 0x380 + y%48;
    else:
     print('unexpected bank number')
    ASIC.RowCounter.set(x) #6011
    ASIC.ColCounter.set(colToWrite) #6013
    readBack = ASIC.WritePixelData.set(value) #5000
    ASIC.CmdPrepForRead() #0000
    return readBack

In [ ]:
def setPixel (ASIC, Preamp = 4, Vld1_b = 1, Pixel_CB = 4, Filter_DAC = 38, VRef_DAC = 53):
    ASIC.Preamp.set(Preamp)
    ASIC.Vld1_b.set(Vld1_b)
    ASIC.Pixel_CB.set(Pixel_CB)
    ASIC.Filter_DAC.set(Filter_DAC)
    ASIC.VRef_DAC.set(VRef_DAC)


    print('Preamp', ASIC.Preamp.get())
    print('Vld1_b',ASIC.Vld1_b.get())
    print('Pixel_CB',ASIC.Pixel_CB.get())
    print('Filter_DAC',ASIC.Filter_DAC.get())
    print('VRef_DAC',ASIC.VRef_DAC.get())

In [1]:
def run_asic_test(DATADEBUG, numFrames = 4000, initialDelay = 25.0):

    # damage control
    DATADEBUG.enableDataDebug(True)
    DATADEBUG.cleanData()
    TRIG.StopTriggers()
    
    #sets TriggerRegisters
    TRIG.AcqCountReset()
    TRIG.numberTrigger.set(numFrames)
    TRIG.AutoTrigPeriod.set(50000) #in steps of 10ns
    TRIG.AutoRunEn.set(True)
    TRIG.RunTriggerEnable.set(True)
    #wait for the camera to get thermally stable
    time.sleep(initialDelay)    
    TRIG.AutoDaqEn.set(True)
    TRIG.DaqTriggerEnable.set(True)

    while (TRIG.AcqCount.get() != numFrames) :
        print("Triggers sent: {}".format(TRIG.AcqCount.get()) , end='\r')
        time.sleep(0.1)

    # stops triggers
    TRIG.StopTriggers()
    time.sleep(0.1)

    while ( numFrames > DATADEBUG.getData().shape[2]) :
        time.sleep(1)
        print("Descrambled {}".format(DATADEBUG.getData().shape[2]), end='\r')
    print("Data Descrambled")
    print(DATADEBUG.getData().shape)

    # Print the status
    DeltaAcqCount = TRIG.AcqCount.get()
    DeltaDaqCount = TRIG.DaqCount.get()
    
    print( f'Delta Aquisition Counter: {DeltaDaqCount}')
    print( f'Number of frames received: {DATADEBUG.getData().shape}' )

    return DATADEBUG.getData()


In [ ]:
def read_asic_test(numFrames = 4000):
    print(filename)
    # Setup and open the file writer
   
    ########################################################################################################################3
    f = open(filename, mode = 'rb')
    imgDesc = []
    for i in range(int(numFrames/MAX_NUMBER_OF_FRAMES_PER_BATCH)-1):
        print("Starting to get data set %d" % (i))
        allFrames = currentCam.getData(f,8)
        imgDesc2 = currentCam.getDescImaData(allFrames[:,:-8])
        if i == 0:
            headers = allFrames[:,0:6]
            imgDesc = imgDesc2
        if i == 1: #skips the first dataset
            headers = allFrames[:,0:6]
            imgDesc = imgDesc2
        else:
            headers = np.concatenate((headers, allFrames[:,0:6]),0)
            imgDesc = np.concatenate((imgDesc, imgDesc2),0)
        if allFrames.shape[0] != MAX_NUMBER_OF_FRAMES_PER_BATCH:
            break


    numberOfFrames = allFrames.shape[0]
    print(allFrames.shape)
    ###########################################################################################################################
    return imgDesc

In [ ]:
def complete_test(filename):
    ########################################################################################################################
    if os.path.exists(filename):
       os.remove(filename)
       print("The file %s was removed" %(filename))
    else:
       print("The file does not exist")
    imgDesc = run_asic_test(4000)
    np.save(filename, imgDesc)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0,imgDesc=imgDesc)
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()

    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    #plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()

    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    #plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    noise_print_roi(x=65,y=134,width=40,height=10, imgDesc=imgDesc)    
    
    noise_print_roi(x=10,y=10,width=50,height=50, imgDesc=imgDesc)
    
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(imgDesc[100,:,0:192], cmap='gray', vmin=5000, vmax=7000, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)
    
    ########################################################################################################################
    imgDesc = read_asic_test(20000)
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0, imgDesc=imgDesc)
    #scatter plot
    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,10,10]
    plt.plot(dataset, 'rx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()


    plt.figure(1,figsize=(12,8),dpi=150)
    dataset = imgDesc[:,140,75]
    plt.plot(dataset, 'bx', label=f'\somelabel') 
    plt.title("baseline") 
    plt.legend()
    plt.show()
    
    plt.figure(2,figsize=(12,8),dpi=150)
    plt.hist(dataset.reshape(-1), bins=100, label=f'\somelabel') 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()

    
    noise_print_roi(x=10,y=10,width=50,height=50,imgDesc=imgDesc)
    
    return imgDesc

In [1]:
def noise_print(offset = 0, imgDesc=[]):
    print(imgDesc[100].shape)
    hg_n  = imgDesc[:,:,offset:offset+192].astype('float64')
    hg_n -= np.median(hg_n, axis=0, keepdims=True)
    hg_n -= np.median(hg_n, axis=(1,2), keepdims=True)
    hg_n -= np.median(hg_n, axis=2, keepdims=True) # rows (cols)
    hg_n -= np.median(hg_n, axis=1, keepdims=True) # cols (rows)
    imgDescCMC = hg_n
    noiseMap = np.std(imgDescCMC,0)
    darkAvg  = np.mean(imgDesc,0)
    
    #noiseMap = np.std(imgDesc[:,:,offset:offset+192],0)
    #darkAvg  = np.mean(imgDesc[:,:,192:384],0)
    medianNoise = np.median(noiseMap)
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(noiseMap, cmap='gray', vmin=0, vmax=50)
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

    plt.figure(2,figsize=(6,4),dpi=150)
    dataset = noiseMap
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level w/o CMC %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    #plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    print()
    return noiseMap, darkAvg

def noise_print_roi(x=0,y=0,width=144,height=192,imgDesc=[]):
    print(imgDesc[100].shape)
    hg_n  = imgDesc[:,y:y+height,x:x+width].astype('float64')
    hg_n -= np.median(hg_n, axis=0, keepdims=True)
    hg_n -= np.median(hg_n, axis=(1,2), keepdims=True)
    hg_n -= np.median(hg_n, axis=2, keepdims=True) # rows (cols)
    hg_n -= np.median(hg_n, axis=1, keepdims=True) # cols (rows)
    imgDescCMC = hg_n
    noiseMap = np.std(imgDescCMC,0)
    darkAvg  = np.mean(imgDesc,0)
    
    #noiseMap = np.std(imgDesc[:,:,offset:offset+192],0)
    #darkAvg  = np.mean(imgDesc[:,:,192:384],0)
    medianNoise = np.median(noiseMap)
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(noiseMap, cmap='gray', vmin=-50, vmax=50, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

    plt.figure(2,figsize=(6,4),dpi=150)
    dataset = noiseMap
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level w/o CMC %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    print()
    return noiseMap, darkAvg

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [1]:
def glitchFinder_print(x=0,y=0,width=144,height=192):
    print(imgDesc[100].shape)
    hg_n  = imgDesc[:,y:y+height,x:x+width].astype('float64')
    imgDescCMC = hg_n
    noiseMap = np.std(imgDescCMC,0)
    darkAvg  = np.mean(imgDescCMC,0)
    
    medianNoise = np.median(noiseMap)
    refDarkValue = int(np.mean(np.mean(darkAvg)))
    
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(imgDescCMC[0], cmap='gray', vmin=refDarkValue-50, vmax=refDarkValue+50, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

    plt.figure(2,figsize=(6,4),dpi=150)
    dataset = imgDescCMC
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level w/o CMC %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(imgDescCMC[0]-darkAvg, cmap='gray', vmin=-50, vmax=50, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)
    
    plt.figure(2,figsize=(6,4),dpi=150)
    dataset = imgDescCMC - darkAvg 
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level w/o CMC %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    
    return noiseMap, darkAvg